<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_4%EC%9D%BC%EC%B0%A8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [ ]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2 MB 35 kB/s 
     |████████████████████████████████| 198 kB 19.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=3bf3ba3aa5a8e5895fcaa7c312e820edb316aa92485da0b6999b53dfef9c7970
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [ ]:
spark

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2021-07-23 14:29:24--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.253.17
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.253.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.06s   

2021-07-23 14:29:25 (566 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [ ]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Jul 16 13:20 sample_data
-rw-r--r-- 1 root root 36240 Jan 31 01:46 boston_housing.csv


In [ ]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [ ]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [ ]:
data_2 = assembler.transform(data)

In [ ]:
data_2.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [ ]:
train, test = data_2.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

## 모델 성능 측정

In [ ]:
evaluation_summary = model.evaluate(test)

In [ ]:
evaluation_summary

In [ ]:
evaluation_summary.meanAbsoluteError

2.9835809173454946

In [ ]:
evaluation_summary.rootMeanSquaredError

4.064639789667804

In [ ]:
evaluation_summary.r2

0.7438812097739442

## 모델 예측값 살펴보기 

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+----+-----+----+-----+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+-----+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01381|80.0| 0.46|   0|0.422|7.875|32.0| 5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...|40.622133425847935|
|0.01709|90.0| 2.02|   0| 0.41|6.728|36.1|12.1265|  5|187|   17.0|384.46|  4.5|30.1|[0.01709,90.0,2.0...|25.781742842475545|
| 0.0187|85.0| 4.15|   0|0.429|6.516|27.7| 8.5353|  4|351|   17.9|392.43| 6.36|23.1|[0.0187,85.0,4.15...|25.258198351285507|
|0.02729| 0.0| 7.07|   0|0.469|7.185|61.1| 4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|30.696243335944626|
|0.02985| 0.0| 2.18|   0|0.458| 6.43|58.7| 6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|25.613102065511093|


In [ ]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|50.0|[0.01381,80.0,0.4...|40.622133425847935|
|30.1|[0.01709,90.0,2.0...|25.781742842475545|
|23.1|[0.0187,85.0,4.15...|25.258198351285507|
|34.7|[0.02729,0.0,7.07...|30.696243335944626|
|28.7|[0.02985,0.0,2.18...|25.613102065511093|
|31.2|[0.03049,55.0,3.7...| 28.40812175125548|
|17.5|[0.03113,0.0,4.39...|16.524961528703344|
|20.6|[0.03306,0.0,5.19...| 22.24790744553615|
|19.5|[0.03427,0.0,5.19...|20.326679619822244|
|28.5|[0.03502,80.0,4.9...| 33.55194642363187|
|20.9|[0.03548,80.0,3.6...| 21.65432096471288|
|23.5|[0.03584,80.0,3.3...|30.412396980225278|
|27.9|[0.03615,80.0,4.9...|31.904688419609407|
|22.0|[0.03932,0.0,3.41...|27.598362896695328|
|22.9|[0.04203,28.0,15....|  29.0495258852579|
|20.6|[0.04527,0.0,11.9...|22.605009330072093|
|23.3|[0.0456,0.0,13.89...| 27.87532449226261|
|22.3|[0.0459,52.5,5.32...|27.420507995384483|
|30.3|[0.0466

In [ ]:
model.save("boston_housing_model")

In [ ]:
!ls boston_housing_model

data  metadata


In [ ]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Jul 22 09:11 data
drwxr-xr-x 2 root root 4096 Jul 22 09:11 metadata


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model2" 
model.save(path)

In [ ]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [ ]:
predictions2 = loaded_model.transform(test)

In [ ]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|50.0|[0.01381,80.0,0.4...|40.622133425847935|
|30.1|[0.01709,90.0,2.0...|25.781742842475545|
|23.1|[0.0187,85.0,4.15...|25.258198351285507|
|34.7|[0.02729,0.0,7.07...|30.696243335944626|
|28.7|[0.02985,0.0,2.18...|25.613102065511093|
|31.2|[0.03049,55.0,3.7...| 28.40812175125548|
|17.5|[0.03113,0.0,4.39...|16.524961528703344|
|20.6|[0.03306,0.0,5.19...| 22.24790744553615|
|19.5|[0.03427,0.0,5.19...|20.326679619822244|
|28.5|[0.03502,80.0,4.9...| 33.55194642363187|
|20.9|[0.03548,80.0,3.6...| 21.65432096471288|
|23.5|[0.03584,80.0,3.3...|30.412396980225278|
|27.9|[0.03615,80.0,4.9...|31.904688419609407|
|22.0|[0.03932,0.0,3.41...|27.598362896695328|
|22.9|[0.04203,28.0,15....|  29.0495258852579|
|20.6|[0.04527,0.0,11.9...|22.605009330072093|
|23.3|[0.0456,0.0,13.89...| 27.87532449226261|
|22.3|[0.0459,52.5,5.32...|27.420507995384483|
|30.3|[0.0466